In [ ]:
# run_local_smoketest_no_multiproc.py
import numpy as np

from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models
from masked_ppo_wrapper import DiscreteActionSetWrapper, mask_fn
from sb3_contrib.common.wrappers.action_masker import ActionMasker

BASE_DIR = "../data/Fantasy-Premier-League"
SEASONS  = ["2020-21", "2021-22", "2022-23", "2023-24"]  # pick a couple you want to sanity-check

def make_env_for_season(season: str):
    env = FPLEnv(
        load_season_fn=load_season_fn,
        load_gw_fn=load_gw_fn,
        seasons=[season],
        base_dir=BASE_DIR,
        start_gw=2,
        budget=100.0,
        temperature=0.8,
        transfer_hit=-4.0,
        max_free_transfers=5,
        models=load_models("../models/rand_forest/classifiers"),
    )
    # 1) compact (out,in)->Discrete
    env = DiscreteActionSetWrapper(env)
    # 2) masking wrapper (do not call mask_fn on this; call it on the wrapped base below)
    env = ActionMasker(env, mask_fn)
    return env

def get_base(env):
    """Unwrap ActionMasker -> DiscreteActionSetWrapper for mask_fn & _choices access."""
    return getattr(env, "env", env)

def run_one_season(season: str, rng: np.random.Generator):
    env = make_env_for_season(season)

    # Prefer reset(options=...) if supported; otherwise patch the window
    try:
        obs, info = env.reset(options={"force_window": (1, 38)})
    except TypeError:
        obs, info = env.reset()
        env.unwrapped.start_gw_actual = 1
        env.unwrapped.end_gw = 38
        env.unwrapped.current_gw = max(getattr(env.unwrapped, "current_gw", 2), 2)

    # predictor/xP must be attached
    pred = env.unwrapped.season_ctx.get("predictor", None)
    assert pred is not None, f"{season}: predictor missing (did you load models?)"

    base = get_base(env)  # DiscreteActionSetWrapper

    # mask must have legal actions
    mask = mask_fn(base)
    assert np.any(mask), f"{season}: initial action mask is all zeros"

    done = truncated = False
    steps = 0
    gws_finished = 0
    prev_phase = info.get("phase", getattr(env.unwrapped, "phase", 1))

    while not (done or truncated):
        # random legal action (sample from mask computed on the base wrapper)
        mask = mask_fn(base)
        legal = np.flatnonzero(mask)
        assert legal.size > 0, f"{season}: mask all zeros during rollout"
        action = int(rng.choice(legal))

        obs, reward, done, truncated, info = env.step(action)
        steps += 1

        # detect GW boundary: phase toggles 2 -> 1 after phase-2 step
        cur_phase = info.get("phase", getattr(env.unwrapped, "phase", 1))
        if prev_phase == 2 and cur_phase == 1:
            gws_finished += 1
        prev_phase = cur_phase

    assert gws_finished == 38, f"{season}: expected 38 GWs, got {gws_finished} (steps={steps})"
    print(f"{season}: ✅ ran GW1→GW38 with random legal actions (steps={steps})")
    env.close()

def main():
    rng = np.random.default_rng(0)
    for season in SEASONS:
        try:
            run_one_season(season, rng)
        except Exception as e:
            print(f"{season}: ❌ {e}")



2020-21: ❌ 2020-21: expected 38 GWs, got 11 (steps=22)
2021-22: ❌ 2021-22: expected 38 GWs, got 12 (steps=24)
2022-23: ❌ 2022-23: expected 38 GWs, got 12 (steps=24)
2023-24: ❌ 2023-24: expected 38 GWs, got 12 (steps=24)


In [5]:
env = FPLEnv(
    load_season_fn=load_season_fn,
    load_gw_fn=load_gw_fn,
    seasons=['2024-25'],  # specify the season you want to test
    base_dir=BASE_DIR,
    start_gw=2,
    budget=100.0,
    temperature=0.8,
    transfer_hit=-4.0,
    max_free_transfers=5,
    models=load_models("../models/rand_forest/classifiers"),
)
env.reset()
env.squad.players

[Player(pid=383, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.2, features=(4.314225396497191, 2.840979613330805, 4.0, 3.5, 0.0, 0.7705448483990107, 100.0), pooling_metric=70726.0, xP=0.497496078396955),
 Player(pid=521, name='Łukasz Fabiański', pos='GK', team='West Ham', team_id=19, price=4.1, features=(3.35562044980355, 2.2865044329436275, 3.0, 3.0, 0.0, 0.7445162196320741, 100.0), pooling_metric=53017.0, xP=0.6102491340504009),
 Player(pid=70, name='Milos Kerkez', pos='DEF', team='Bournemouth', team_id=3, price=4.6, features=(5.47949545499026, 5.749739030576289, 2.0, 2.75, 0.0, 0.9809542306975786, 100.0), pooling_metric=247482.0, xP=0.7024796009240107),
 Player(pid=18, name='William Saliba', pos='DEF', team='Arsenal', team_id=1, price=6.1, features=(5.997746633357021, 4.146851328174412, 3.0, 2.75, 0.0, 0.934927371788298, 100.0), pooling_metric=282622.0, xP=0.6371175762175386),
 Player(pid=6, name='Jurriën Timber', pos='DEF', team='Arsenal', team_id=1, price=5.6, 